In [2]:
from sunpy.map import Map
import matplotlib.pyplot as plt
import numpy as np
import astropy.units as u
import pandas as pd
from glob import glob
import re
from astropy.coordinates import SkyCoord
import copy

In [3]:
flare_location_df = pd.read_csv('010121-130223_final_filtered_flare_list.csv', index_col=0)

In [4]:
flare_location_df['flare location test'] = 'not tested'

In [5]:
flare_location_df['request ID']

0       2101060003
1       2101250007
2       2102130001
3       2102140006
4       2102140007
           ...    
9441    2302124344
9442    2302124344
9443    2302124344
9444    2302124344
9445    2302124344
Name: request ID, Length: 9446, dtype: int64

In [7]:
flare_location_df.head()

,flare_id,att_in,GOES_flux,start_UTC,peak_UTC,end_UTC,4-10 keV,10-15 keV,number_available_files,available_file_request_IDs,...,Y flare loc,FWDFIT X SIGMA,FWDFIT Y SIGMA,MAX X COORD BP,MAX Y COORD BP,Lon Heliographic,Lat Heliographic,Earth X Helioprojective,Earth Y Helioprojective,flare location test
0,2101061410,False,1.325816e-08,2021-01-06T14:08:15.464,2021-01-06 14:10:47.465,2021-01-06T14:21:31.466,671,151,1,[2101060003],...,-435.479000,2.529080,2.698680,-37.076300,-437.500000,134.072058,-18.944095,660.921841,-356.033488,not tested
1,2101250635,False,5.092464e-07,2021-01-25T06:32:51.072,2021-01-25 06:35:31.073,2021-01-25T06:41:51.073,607,91,1,[2101250007],...,676.563354,0.831036,1.331697,-1651.377075,667.769958,71.481742,22.347076,855.563671,396.705104,not tested
2,2102132346,False,1.535086e-08,2021-02-13T23:45:06.658,2021-02-13 23:46:18.658,2021-02-13T23:49:54.659,607,83,1,[2102130001],...,715.233521,1.172359,0.926795,637.098267,705.708862,-139.159952,22.861221,-583.288612,293.652991,not tested
3,2102140144,False,1.557456e-08,2021-02-14T01:41:06.670,2021-02-14 01:44:14.670,2021-02-14T01:49:14.671,1983,463,1,[2102140006],...,706.836792,0.826290,0.892662,617.354980,705.548523,-139.906750,22.613293,-575.454512,288.716023,not tested
4,2102140529,False,1.816594e-08,2021-02-14T05:23:46.693,2021-02-14 05:29:06.693,2021-02-14T05:36:18.694,671,83,1,[2102140007],...,727.776001,1.512266,1.221617,695.439880,724.824646,-136.497375,23.276099,-612.172284,303.481826,not tested


In [12]:
Map.__doc__

"\n    A factory for generating coordinate aware 2D images.\n\n    This factory takes a variety of inputs, such as file paths, wildcard\n    patterns or (data, header) pairs.\n\n    Depending on the input different return types are possible.\n\n    Parameters\n    ----------\n    \\*inputs\n        Inputs to parse for map objects. See the examples section for a\n        detailed list of accepted inputs.\n\n    sequence : `bool`, optional\n        Return a `sunpy.map.MapSequence` object comprised of all the parsed maps.\n\n    composite : `bool`, optional\n        Return a `sunpy.map.CompositeMap` object comprised of all the parsed maps.\n\n    Returns\n    -------\n    `sunpy.map.GenericMap`\n        If the input results in a singular map object, then that is returned.\n\n    `list` of `~sunpy.map.GenericMap`\n        If multiple inputs are given and ``sequence=False`` and ``composite=False``\n        (the default) then a list of `~sunpy.map.GenericMap` objects will be\n        returne

In [15]:
%matplotlib inline
filenamelist = glob('./bp_maps3/*.fits')
filenamelist.sort()
bp_maps = Map(filenamelist, silence_errors=True)
bp_maps_copy = copy.deepcopy(bp_maps)

for i in range(len(bp_maps)):
    m = re.search('bp_nat_map-(.+?).fits', filenamelist[i])
    req_id = m.group(1)
    if len(flare_location_df[flare_location_df['request ID'] == int(req_id)]) != 0:
        
        loc_x = flare_location_df[flare_location_df['request ID'] == int(req_id)]['MAX X COORD BP'][flare_location_df[flare_location_df['request ID'] == int(req_id)]['MAX X COORD BP'].index[-1]]
        loc_y = flare_location_df[flare_location_df['request ID'] == int(req_id)]['MAX Y COORD BP'][flare_location_df[flare_location_df['request ID'] == int(req_id)]['MAX Y COORD BP'].index[-1]]
        
        flare_id = flare_location_df[flare_location_df['request ID'] == int(req_id)]['flare_id'].iloc[-1]
        
        pix_x = int(bp_maps[i].world_to_pixel(SkyCoord(loc_x*u.arcsec, loc_y*u.arcsec, frame=bp_maps[i].coordinate_frame))[1].value)
        pix_y = int(bp_maps[i].world_to_pixel(SkyCoord(loc_x*u.arcsec, loc_y*u.arcsec, frame=bp_maps[i].coordinate_frame))[0].value)
        
        original_max_val = bp_maps[i].data[pix_x,pix_y]

        bp_maps_copy[i].data[pix_x-20:pix_x+20,pix_y-20:pix_y+20] = 0
        
        new_max_val = bp_maps_copy[i].data.max()
                
        if new_max_val < 0.9*original_max_val:
            flare_location_df[(flare_location_df['request ID'] == int(req_id)) & (flare_location_df['flare_id'] == int(flare_id))] = flare_location_df[(flare_location_df['request ID'] == int(req_id)) & (flare_location_df['flare_id'] == int(flare_id))].replace('not tested', 'location is reliable')
          

        else:
            flare_location_df[(flare_location_df['request ID'] == int(req_id)) & (flare_location_df['flare_id'] == int(flare_id))] = flare_location_df[(flare_location_df['request ID'] == int(req_id)) & (flare_location_df['flare_id'] == int(flare_id))].replace('not tested', 'unreliable location, check manually')



OSError: Failed to read bp_maps3/bp_nat_map-2209022718.fits

In [16]:
flare_location_df.head()

,Unnamed: 0,flare_id,att_in,GOES_flux,start_UTC,peak_UTC,end_UTC,4-10 keV,10-15 keV,number_available_files,...,request ID,X flare loc,Y flare loc,FWDFIT X SIGMA,FWDFIT Y SIGMA,MAX X COORD BP,MAX Y COORD BP,Earth X,Earth Y,flare location test
0,0,2211090050,False,6.073233e-07,2022-11-09T00:44:44.012,2022-11-09 00:50:20.013,2022-11-09T00:52:56.013,1215,271,1,...,2211090305,-15.344300,181.459000,0.918372,1.007100,-16.415800,180.574000,-382.781296,187.115997,pass
1,3,2211090720,False,9.645386e-07,2022-11-09T07:18:12.052,2022-11-09 07:20:44.052,2022-11-09T07:28:40.053,2431,247,1,...,2211091049,359.304382,523.129578,0.484560,0.617333,359.168732,522.427246,-133.265314,381.172753,pass
2,4,2211090908,False,6.582638e-07,2022-11-09T08:58:40.062,2022-11-09 09:08:44.063,2022-11-09T09:14:56.064,607,135,1,...,2211096407,377.920044,529.196899,1.762542,2.026066,374.929199,529.791260,-119.780634,384.756518,pass
3,5,2211090958,False,6.191904e-07,2022-11-09T09:55:28.068,2022-11-09 09:58:44.069,2022-11-09T10:06:04.069,607,75,1,...,2211091409,824.235107,503.754547,1.559922,2.214852,822.638184,496.840912,170.809223,354.432526,pass
4,6,2211091013,False,6.162408e-07,2022-11-09T10:11:28.070,2022-11-09 10:13:24.070,2022-11-09T10:16:24.070,735,115,1,...,2211091673,407.907379,450.271484,1.241306,1.207590,407.163910,447.880310,-109.159107,338.202067,pass
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
57,68,2211111825,False,8.307466e-07,2022-11-11T18:24:28.016,2022-11-11 18:25:16.016,2022-11-11T18:25:36.016,1599,367,1,...,2211115828,887.013916,173.249420,0.766406,0.466865,887.460876,163.479645,243.915183,163.700184,pass
58,69,2211112006,False,8.048406e-07,2022-11-11T20:00:12.026,2022-11-11 20:06:04.026,2022-11-11T20:12:20.027,607,75,1,...,2211119540,968.164001,222.679474,5.241368,3.643471,964.073792,217.694092,311.034526,190.224313,fail
59,70,2211112147,False,7.720208e-07,2022-11-11T21:43:11.936,2022-11-11 21:47:51.937,2022-11-11T21:56:07.937,735,99,1,...,2211112507,796.479187,252.935638,5.182872,5.007693,791.995361,248.469131,185.604517,217.925378,fail
60,71,2211112226,False,1.154457e-06,2022-11-11T22:18:43.940,2022-11-11 22:26:51.941,2022-11-11T22:45:55.943,1471,115,1,...,2211113044,966.374695,263.555695,0.681046,0.547328,962.348145,263.869659,315.296098,215.948045,pass


In [17]:
len(flare_location_df[flare_location_df['flare location test']=='pass'])/len(flare_location_df) 

0.8548387096774194

In [18]:
len(flare_location_df[flare_location_df['flare location test']=='fail'])/len(flare_location_df) 

0.08064516129032258

In [19]:
len(flare_location_df[flare_location_df['flare location test']=='not tested'])/len(flare_location_df) 

0.06451612903225806

In [20]:
flare_location_df.to_csv('010121-130223_flare_location_df_with_test.csv')
